### Import libraries that we need

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import plotly.express as px
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import scipy
import numpy as np
import scipy.fftpack
from itertools import islice

### Set a list of days to read from directory

In [ ]:
num_of_days = 72000
step = 60
current_day_list = 51480
file_names_list = []
days_to_iterate = current_day_list + num_of_days
    
while current_day_list < days_to_iterate-step:
    current_day_list += step
    file_names_list.append(current_day_list)    

### Read data from directories
Don't forget to set a variable with a number of samples (folders) with experiments in the directory

In [ ]:
number_of_samples = 10
root_path = '/data/home/nsov/eurace/original/reproduce_results/'

qe_list = [1, 0]
# dividents = [0.5, 0.7, 0.9]
dividents = [0.8]
dict_of_dataframes = {}
last_day = max(file_names_list)

for div in dividents:
    case_name = 'test_div_' + str(div)
    dict_of_dataframes[case_name] = pd.DataFrame()
    for sample in range(number_of_samples):
        dir_name = case_name + '_' + str(sample)
        current_dir = root_path + dir_name
        try:
            df = pd.DataFrame()

            for file in file_names_list: 
                if file == 0:
                    xml_data = open(current_dir + '/{}.xml'.format('0_initial'), 'r').read()  # Read file
                else:
                    xml_data = open(current_dir + '/{}.xml'.format(file), 'r').read()  # Read file
                root = ET.XML(xml_data)  # Parse XML

                data = []
                cols = []
                names = []
                for i, child in enumerate(root):
                    try:
                        names.append(child.find('name').text)
                    except:
                        pass
                    if child.tag == 'xagent':
                        if child.find('name').text == 'Eurostat':
                            data.append([subchild.text for subchild in child])
                            if file == last_day:
                                cols = [subchild.tag for subchild in child]
                df2 = pd.DataFrame(data)
                df = pd.concat([df, df2], ignore_index=True)
                if file == last_day:
                    df.columns = cols  # Update column names
            df['case_number'] = [sample] * len(df)
            dict_of_dataframes[case_name] = pd.concat([dict_of_dataframes[case_name], df])
            print('Done for ' + dir_name)
        except:
            print('Error for ' + dir_name)

### Function for sliding window over time series

In [ ]:
def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield sum(result)/n
    for elem in it:
        result = result[1:] + (elem,)
        yield sum(result)/n

### FFT function for time series analysis

In [ ]:
set_window_size = 4 #The size of a sliding window

In [ ]:
def draw_fft_plot(unemployment_series, variable, case):
    mean_of_series = unemployment_series.mean()
    # If you want to use fft with sliding window function to erase high frequency noise uncomment folowing line
    unemployment_window = list(i - unemployment_series.mean()  for i in window(unemployment_series, n=set_window_size))
    # If you want to use fft without sliding window function to leave high frequency noise uncomment folowing line
#     unemployment_window = list(i - unemployment_series.mean()  for i in unemployment_series)
    # If you want to use butterworth filer to erase high frequency noise uncomment folowing line
#     unemployment_window = list(i - mean_of_series  for i in my_butterworth(unemployment_series))

    fft_unemp_complex = np.fft.rfft(unemployment_window)
    fft_unemp = []
    for i in fft_unemp_complex:
        fft_unemp.append(abs(i))
    func_to_draw = fft_unemp
    n = len(unemployment_window)
    
    x = np.fft.rfftfreq(n, d=1)

    fig = go.Figure([
        go.Scatter(
            x=x,
            # You can youse square function to make fft peaks more obvious
#             y=np.square(func_to_draw),
            # Or draw original fft function
            y=func_to_draw,
            line=dict(color='rgb(0,100,80)'),
            mode='lines'
        )
    ])
    fig.update_layout(xaxis_title='Frequency',
                     yaxis_title='Magnitude of the FFT')
    fig.write_html("fft_{variable}_{case}.html".format(variable=variable, case=case))

### Butterworth low pass filter

In [ ]:
def my_butterworth(input_series):
    sos = signal.butter(10, 0.2*len(input_series), 'lp', fs=len(input_series), output='sos')
    filtered = signal.sosfilt(sos, input_series)
    return filtered

#### set dicts of colors for different dividends ratios that will be used for drawing plots

In [ ]:
rgb_line_dict = {0.3: 'rgb(150,0,0)',0.5: 'rgb(150,0,0)', 0.7 : 'rgb(0,150,0)', 0.8 : 'rgb(0,0,150)' }
rgb_error_dict = {0.3: 'rgba(150,0,0,0.2)',0.5: 'rgb(150,0,0)', 0.7 : 'rgba(0,150,0,0.2)', 0.8 : 'rgba(0,0,150,0.2)' }

#### Some variables for choosing what to draw

In [ ]:
number_of_samples = 1
root_path = '/data/home/nsov/eurace/original/reproduce_results/'

qe_list = [1]
# dividents = [0.5, 0.7, 0.9]
dividents = [0.8]

#### This function counts mean and std for a certain serie from dict of dataframes

In [ ]:
def get_series(case_name, input_series, growth_rate=False):
    dict_of_dataframes[case_name][input_series] = pd.to_numeric(dict_of_dataframes[case_name][input_series])
    variable_frame = dict_of_dataframes[case_name][input_series].to_frame()
    variable_series = variable_frame.groupby(variable_frame.index).mean()[input_series]
    
    # FFT analysis has shown that there are two main harmonics of time series:
    #    1 year cycle (main harmonc)
    #    half year cycle 
    # So if we take average value of the main harmonics with half period interval, we can get accurate average value.
    # Here are experiments for 1 harmonic averaging: 2 points with 0.5 year step
    # experiment of averaging over 2 harmonics: 4 points with 0.25 year step
    # and aditional experiment for whole year averaging: 12 points with 1 month step
    
#     # experiment with 3 timestamps averaging
#     unemployment_series = (unemployment_series + unemployment_series[3:].reset_index(drop=True) +
#                            unemployment_series[6:].reset_index(drop=True) + unemployment_series[9:].reset_index(drop=True))/4
#     unemployment_series = unemployment_series.dropna()

# #         experiment with 6 timestamps averaging
#     unemployment_series = (unemployment_series + unemployment_series[6:].reset_index(drop=True))/2
#     unemployment_series = unemployment_series.dropna()
    
    #         experiment with 1 year timestamps averaging
#     unemployment_series_loop = unemployment_series
#     for i in range(1,12):
#         unemployment_series = unemployment_series + unemployment_series_loop[i:].reset_index(drop=True)
#     unemployment_series = unemployment_series/12
#     unemployment_series = unemployment_series.dropna()
    
    variable_series_std = variable_frame.groupby(variable_frame.index).std()[input_series]
    
    if growth_rate == True:
        variable_series = variable_series.diff()/variable_series
        variable_series = growth_rate_series.dropna()
        variable_series_std = variable_series_std.diff()/variable_series
        variable_series_std = variable_series_std.dropna()
    
    return variable_series, variable_series_std

Set variables that you want to track and their names that will be displayed on the plot

In [ ]:
output_variables = ['gdp', 'unemployment_rate', 'cpi', 'average_wage', 'firm_average_productivity_progress', 'employment_rate', 'monthly_investment_value', 'monthly_output']
plot_names = {'gdp' : 'GDP growth rate', 'unemployment_rate': 'Unemployment rate', 'cpi': 'CPI',
             'average_wage': 'Average wage growth rate', 'firm_average_productivity_progress': 'Firm average productivity progress',
             'employment_rate' : 'Employment rate', 'monthly_investment_value': 'Monthly investment value',
             'monthly_output': 'Monthly output'}

### Main counting cell

In [ ]:
df_boxplot = pd.DataFrame()

list_of_lines_gdp = []
stabilization_offset = 25 # the number of iteration for model stabilization. 
                          # We will not use these iterations for counting numerical results

# Dict where series of values will be saved for building plots
dict_of_lines= {}
for i in output_variables:
    dict_of_lines[i] = []

# Iterate dividends values
for div in dividents:
    # This dict will be used for building boxplots
    long_stay_dict = {}
    case_name = 'test_div_' + str(div)
    
    # For each variable compute mean, std and save a list of values for a plot
    for variable in output_variables:
        # For this variables count its growth rate mean and std instead of absolute values
        if variable == 'average_wage' or 'gdp':
            var_mean, var_std = get_series(case_name, variable, growth_rate = True)
        else:
            var_mean, var_std = get_series(case_name, variable)
        
        # Print mean, std and mean of std for variables in the list
        print(str(variable), ' mean: ', "{:.4}".format(var_mean[stabilization_offset:].mean()))
        print(str(variable), ' std_mean: ', "{:.4}".format(var_mean[stabilization_offset:].std()))
        print(str(variable), ' std: ', "{:.4}".format(var_std[stabilization_offset:].mean()))
        
        #Save these values for building boxplots
        long_stay_dict[variable] = [var_mean, var_std]
        
        # Draw interactive html plots with plotly
        x = list(var_mean.index)
        y = list(var_mean)
        y_err_up = list(var_mean + var_std)
        y_err_down = list(var_mean - var_std)

        dict_of_lines[variable].append(
            go.Scatter(
                name=str('Quantitative easing ')+str(div),
                x=x,
                y=pd.Series(i for i in window(y, n=set_window_size)),
                #If you want to use butterworth filter uncomment following line
                #y=my_butterworth(y)
                line=dict(color=rgb_line_dict[div]),
                mode='lines'
            ))
        dict_of_lines[variable].append(
            go.Scatter(
                name=str('Quantitative easing')+' (std) '+str(div),
                x=x+x[::-1], # x, then x reversed
                y=y_err_up+y_err_down[::-1], # upper, then lower reversed
                fill='toself',
                fillcolor=rgb_error_dict[div],
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                showlegend=True
            ))

    #draw fft plot for a variable
    draw_fft_plot(long_stay_dict['unemployment_rate'][0][0:], 'unemployment', str(div))

    df_local_boxplot = pd.DataFrame({'Unemployment': unemployment_series.iloc[stabilization_offset:], 'GDP growth rate': growth_rate_series.iloc[stabilization_offset:], 'Dividents' : [div] * len(unemployment_series.iloc[stabilization_offset:])})
    df_boxplot= pd.concat([df_boxplot, df_local_boxplot], ignore_index=True)

# Pass lists to plotly and build all plots
for variable in dict_of_lines.keys():
    fig = go.Figure(dict_of_lines[variable])
    fig.update_layout(legend_title_text = 'Case',
                     xaxis_title='Month',
                     yaxis_title=plot_names[variable])
    fig.write_html(plot_names[variable]+".html")

### Build boxplots
I was saving datasets with different policies to different datasets.\
Scripts above give all information for one policy case for different dividends payout ratios.\
Thus, if I want to campare results of two different policies I load a dataframe with one policy.\
Then I run main counting cell and save dict of lines to "qe_dict".\
After that I load another policy case, run main counting cell and save data to "std_dict".\
And the I compare these two policies.\
I could do it more elegant way, but I needed this part only once and I'm too lazy to change it:)

In [ ]:
qe_dict = dict_of_lines

In [ ]:
std_dict = dict_of_lines

In [ ]:
dict_of_lines2 = {}
for i in output_variables:
    dict_of_lines2[i] = []

In [ ]:
for i in dict_of_lines:
    dict_of_lines2[i] = []
    for item in qe_dict[i]:
        dict_of_lines2[i].append(item)
    for item in std_dict[i]:
        dict_of_lines2[i].append(item)

In [ ]:
dict_of_lines2.keys()

In [ ]:
standart_dict = long_stay_dict

In [ ]:
quntitative_dict = long_stay_dict

In [ ]:
for i in standart_dict.keys():
    df_local = pd.DataFrame({plot_names[i]: standart_dict[i][0][600:], 'case' : ['No policy'] * len(standart_dict[i][0][600:])})
    df_local = pd.concat([df_local, pd.DataFrame({plot_names[i]: quntitative_dict[i][0][600:], 'case' : ['Quantitative easing'] * len(quntitative_dict[i][0][600:])})], ignore_index=True)
    
    fig = px.box(df_local, x = 'case', y = plot_names[i])
    fig.write_html('bp_' + str(i) +'.html')

## Save/Load Dataframes

##### Save

In [ ]:
full_df = pd.DataFrame()
for key, dataframe in dict_of_dataframes.items():
    dataframe['case_name'] = [key]*len(dataframe)
full_df = pd.concat(dict_of_dataframes.values())
full_df.to_csv('initial_state_300_years_08.csv')

##### Load

In [ ]:
full_df2 = pd.read_csv('standart_2k.csv', index_col= 'Unnamed: 0')  

In [ ]:
dict_of_dataframes = {}
for i in list(full_df2.case_name.unique()):
    dict_of_dataframes[i] = full_df2.loc[full_df2.case_name == i]

## Correlation
#### Does not work anymore!
I thought I would count correlation between two series with different offset one to another, but I don't need to.\
I will leave this part of code just in case if I will change my mind :)

In [ ]:
years = 2
start_position = len(df_corr.unemployment_series)//2
correlation_change = []
for i in range(years*2*12):
    correlation_change.append(np.corrcoef(df_corr.unemployment_series[start_position-years*12+i:start_position+i], df_corr.growth_rate_series[start_position-years*12+i:start_position+i])[0,1])

In [ ]:
x = list(range(len(correlation_change)))

fig = go.Figure([
    go.Scatter(
        x=x,
        y=correlation_change,
        line=dict(color='rgb(0,100,80)'),
        mode='lines'
    )
])
fig.write_html("correlation_change.html")

In [ ]:
years = 1
start_position = len(df_corr.unemployment_series)//4

min_month = []
corr_val = []
for shift in range(start_position*2):
    correlation_change = []
    for i in range(years*2*12):
        correlation_change.append(np.corrcoef(df_corr.unemployment_series[start_position-years*12+i+shift:start_position+i+shift], df_corr.growth_rate_series[start_position-years*12+shift:start_position+shift])[0,1])
    min_month.append(correlation_change.index(min(correlation_change)) - years*12)
    corr_val.append(min(correlation_change))

In [ ]:
fig = px.histogram(x=min_month, nbins = years*2*24)
fig.write_html("correlation_hist.html")